
# Preproccesing



## Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
import gensim
import numpy as np
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary

## Load data

In [ ]:
df_twitter = pd.read_csv("data.csv")

## Review dataset and select column of interest

In [ ]:
df_twitter.head(2)

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media
0,2021-05-15T18:27:35+00:00,1393634179930214401,1393634179930214401,💛❤️ R E S I S T E N C I A 💛💙❤️\r\nQueremos un ...,es,False,False,1,2,0,0,NaN,NaN,NaN,1389389028785631235,Distritoestbog,Distritoestudiantil,False,NaN,NaN,"#ParoNacional , #ParoNacional15M , #ColombiaSO...",NaN,NaN,"[{""url"": ""https://pbs.twimg.com/ext_tw_video_t..."
1,2021-05-15T14:03:51+00:00,1393567807632195584,1393408901299703812,@PaulinaVLackner @Ladelascuentas @RamirezAnlle...,es,False,False,2,0,0,0,NaN,NaN,NaN,702564560608370688,sontohi,Sonia,False,1.393464e+18,606068813.0,#ColombiaDDHH,"PaulinaVLackner , Ladelascuentas , RamirezAnll...",NaN,[]


In [76]:
print(df_twitter['full_text'][3])

No sé dónde es, pero que valentía la de los jóvenes a quienes debería escuchar el estado.
#NosEstanMantando
#ColombiaDDHH 
@nytimes @UnivisionNews @bbcmundo @CNN @ABC  @CorteIDH @ONU_derechos @AmnistiaOnline @ONUHumanRights @cerosetenta @ONU_derechos  @CIDH  @TembloresOng https://t.co/ok6OAjCCBh


In [77]:
df_text = df_twitter['full_text']
df_text = df_text.to_frame()

In [89]:
df_text.head(5)

,full_text
0,💛❤️ R E S I S T E N C I A 💛💙❤️\r\nQueremos un ...
1,@PaulinaVLackner @Ladelascuentas @RamirezAnlle...
2,@RamirezAnllel abuso del poder #ColombiaDDHH \...
3,"No sé dónde es, pero que valentía la de los jó..."
4,@edladino_ @jclopezcastri @intiasprilla @Pizar...


## Preprocessing



We defined a list of custom words to be exclude from our dataset


In [90]:
black_list = ['más', 'mas', 'unir', 'paises', 'pais', 'espa', 'no', 'os', 'a', 'compa', 'acompa', 'off', 'and', 'grecia', 'the','it', 'to',
              'd',  'et',  'dame',  'il',  'dans', 'that',  'as',   'for',  'it',  'elections',  'would',  'this',  'with', 'york', 'obama', 'chavez', 'gadafi']



Create the cleaner function to clean the spanish text, remove non alpha numeric characters, remove duplicate, remove spanish accutes, remove digits


In [101]:
def cleaner(word):
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', word, flags=re.MULTILINE)
  word = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", word)
  word = re.sub(r'ee.uu', 'eeuu', word)
  word = re.sub(r'\#\.', '', word)
  word = re.sub(r'\n', '', word)
  word = re.sub(r',', '', word)
  word = re.sub(r'\-', ' ', word)
  word = re.sub(r'\.{3}', ' ', word)
  word = re.sub(r'a{2,}', 'a', word)
  word = re.sub(r'é{2,}', 'é', word)
  word = re.sub(r'i{2,}', 'i', word)
  word = re.sub(r'ja{2,}', 'ja', word) 
  word = re.sub(r'á', 'a', word)
  word = re.sub(r'é', 'e', word)
  word = re.sub(r'í', 'i', word)
  word = re.sub(r'ó', 'o', word)
  word = re.sub(r'ú', 'u', word)  
  word = re.sub('[^a-zA-Z]', ' ', word)
  list_word_clean = []
  for w1 in word.split(" "):
    if  w1.lower() not in stopwords:
      list_word_clean.append(w1.lower())

  bigram_list = bigram[list_word_clean]
  out_text = lemmatization(" ".join(bigram_list))
  return out_text




Create the function for select only nouns for our data, this way we are removing adverb, adjetives, verbs, etc. This is doing with spacy


In [102]:
def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = [ token.text for token in nlp(texts) if token.pos_ in 
                 allowed_postags and token.text not in black_list and len(token.text)>2]
    return texts_out
    

In [100]:
len(df_text)

699



Then we look for bigram in our data, i.e pair of words that together have more meanful for our model


In [ ]:
%%time
bigram = gensim.models.Phrases(df_text.full_text.to_list())

In [ ]:
bigram[df_text.full_text.to_list()[0].split()]

In [152]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

*Let's clean all the text**

In [153]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('spanish'))
additional_stopwords=set(black_list)
stopwords = stop.union(additional_stopwords)



We are using Spacy For lemmatize spanish words



In [ ]:
!python -m spacy download es_core_news_md

In [123]:
import spacy
nlp = spacy.load('/usr/local/lib/python3.7/dist-packages/es_core_news_md/es_core_news_md-2.2.5')

In [154]:
df_text.full_text[3]

'No sé dónde es, pero que valentía la de los jóvenes a quienes debería escuchar el estado.\r\n#NosEstanMantando\r\n#ColombiaDDHH \r\n@nytimes @UnivisionNews @bbcmundo @CNN @ABC  @CorteIDH @ONU_derechos @AmnistiaOnline @ONUHumanRights @cerosetenta @ONU_derechos  @CIDH  @TembloresOng https://t.co/ok6OAjCCBh'

In [155]:
cleaner(df_text.full_text[3])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['valentia']

In [156]:
%%time
df_text['cleaned'] = df_text.full_text.apply(cleaner)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 8.2 s, sys: 20.2 ms, total: 8.22 s
Wall time: 8.22 s


In [159]:
df_text

,full_text,cleaned
0,💛❤️ R E S I S T E N C I A 💛💙❤️\r\nQueremos un ...,[futuro]
1,@PaulinaVLackner @Ladelascuentas @RamirezAnlle...,"[abuso, poder]"
2,@RamirezAnllel abuso del poder #ColombiaDDHH \...,[poder]
3,"No sé dónde es, pero que valentía la de los jó...",[valentia]
4,@edladino_ @jclopezcastri @intiasprilla @Pizar...,"[abuso, poder]"
...,...,...
694,@VenomRen @ARMANDO48997112 @Antonio29058651 @H...,[cuentas]
695,@QuinteroCalle Sigan permitiendo estas barbari...,[barbaridades]
696,@OJOVENE @jcy126kerubin @HUS07777781 @Samurai1...,"[espacio, tuit, ataque, miseria, modo]"
697,@OJOVENE @jcy126kerubin @Fredypm @Florencia901...,[combate]




Now we need to build the corpus and the dictionary that gensim need to work, to do that we need to pass a list of list of tokens


In [160]:
dictionary = Dictionary(df_text['cleaned'].to_list())
dictionary.compactify()
# Filter extremes
dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in df_text['cleaned'].to_list()]

In [ ]:
corpus

## NLTK Preprocessing (not developed)

In [87]:
text = "Esta es una prueba de token con nltk objeto con tíldes o tros 45 www.google.com #@ 321 de la casa"
text_token = word_tokenize(text)

In [88]:
text_token

['Esta',
 'es',
 'una',
 'prueba',
 'de',
 'token',
 'con',
 'nltk',
 'objeto',
 'con',
 'tíldes',
 'o',
 'tros',
 '45',
 'www.google.com',
 '#',
 '@',
 '321',
 'de',
 'la',
 'casa']

In [ ]:
token_wo_sw = [word for word in text_token if not word in stopwords.words('spanish')]
print(token_wo_sw)

['Esta', 'prueba', 'token', 'nltk', 'objeto', 'tíldes', 'tros', '45', 'www.google.com', '#', '@', '321', 'casa']


## Preprocessing in sentiment analysis example (not tested)

In [145]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))
stemmer = SnowballStemmer('spanish')
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed
def tokenize(text):
    text = ''.join([c for c in text if c not in non_words])
    tokens =  word_tokenize(text)
    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems


In [146]:
tokenize(text)

['esta',
 'es',
 'una',
 'prueb',
 'de',
 'tok',
 'con',
 'nltk',
 'objet',
 'con',
 'tild',
 'o',
 'tros',
 'wwwgooglecom',
 'de',
 'la',
 'cas']